In [2]:
import pandas as pd

interactions = pd.read_csv("data/interactions_train.csv")
items = pd.read_csv("data/items.csv")

print("Interactions:", interactions.shape)
print("Items:", items.shape)

interactions.head(), items.head()

Interactions: (87047, 3)
Items: (15291, 6)


(      u      i             t
 0  4456   8581  1.687541e+09
 1   142   1964  1.679585e+09
 2   362   3705  1.706872e+09
 3  1809  11317  1.673533e+09
 4  4384   1323  1.681402e+09,
                                                Title  \
 0  Classification décimale universelle : édition ...   
 1  Les interactions dans l'enseignement des langu...   
 2  Histoire de vie et recherche biographique : pe...   
 3  Ce livre devrait me permettre de résoudre le c...   
 4                    Les années glorieuses : roman /   
 
                      Author  \
 0                       NaN   
 1  Cicurel, Francine, 1947-   
 2                       NaN   
 3     Mazas, Sylvain, 1980-   
 4   Lemaitre, Pierre, 1951-   
 
                                           ISBN Valid     Publisher  \
 0                          9782871303336; 2871303339   Ed du CEFAL   
 1                          9782278058327; 2278058320        Didier   
 2                          2343190194; 9782343190198   L'Harmattan 

In [1]:
import os
os.chdir("..")   
os.getcwd()



'd:\\Courses\\DataScience\\AI_Librarian\\Librarian_Recommender_DSML'